### NLP 

- Entity recognition.
- The out should be a df with columns: [document, entities]
     
Datasets:

https://www.kaggle.com/kashnitsky/news-about-major-cryptocurrencies-20132018-40k

Author: Juan Felix Villalobos

In [1]:
import pandas as pd
import numpy as np

import re
from nltk.corpus import stopwords
from nltk.tokenize import WordPunctTokenizer
tok = WordPunctTokenizer()
stop_words = set(stopwords.words('english'))

import nltk
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag
from nltk.chunk import ne_chunk


from nltk.chunk import conlltags2tree, tree2conlltags
from pprint import pprint


%matplotlib inline

In [2]:
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to
[nltk_data]    |     C:\Users\juanf\AppData\Roaming\nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to
[nltk_data]    |     C:\Users\juanf\AppData\Roaming\nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     C:\Users\juanf\AppData\Roaming\nltk_data...
[nltk_data]    |   Package biocreative_ppi is already up-to-date!
[nltk_data]    | Downloading package brown to
[nltk_data]    |     C:\Users\juanf\AppData\Roaming\nltk_data...
[nltk_data]    |   Package brown is already up-to-date!
[nltk_data]    | Downloading package brown_tei to
[nltk_data]    |     C:\Users\juanf\AppData\Roaming\nltk_data...
[nltk_data]    |   Package brown_tei is already up-to-date!
[nltk_data]    | Downloading package cess_cat to
[nltk_data]    |     C:\Users\j

[nltk_data]    |   Package twitter_samples is already up-to-date!
[nltk_data]    | Downloading package udhr to
[nltk_data]    |     C:\Users\juanf\AppData\Roaming\nltk_data...
[nltk_data]    |   Package udhr is already up-to-date!
[nltk_data]    | Downloading package udhr2 to
[nltk_data]    |     C:\Users\juanf\AppData\Roaming\nltk_data...
[nltk_data]    |   Package udhr2 is already up-to-date!
[nltk_data]    | Downloading package unicode_samples to
[nltk_data]    |     C:\Users\juanf\AppData\Roaming\nltk_data...
[nltk_data]    |   Package unicode_samples is already up-to-date!
[nltk_data]    | Downloading package universal_treebanks_v20 to
[nltk_data]    |     C:\Users\juanf\AppData\Roaming\nltk_data...
[nltk_data]    |   Package universal_treebanks_v20 is already up-to-
[nltk_data]    |       date!
[nltk_data]    | Downloading package verbnet to
[nltk_data]    |     C:\Users\juanf\AppData\Roaming\nltk_data...
[nltk_data]    |   Package verbnet is already up-to-date!
[nltk_data]    | 

True

In [3]:
#Loading data

df = pd.read_csv('crypto_news_parsed_2013-2017_train.csv', index_col=0)
df.reset_index(inplace=True)

df = df.head(500)

df.head(3)

,url,title,text,html,year,author,source
0,https://www.ccn.com/bitcoin-price-update-will-...,Bitcoin Price Update: Will China Lead us Down?,Bitcoin Priced in USD on Mt. GoxAbove is Gox p...,"<figure id=""attachment_4090"" style=""width: 838...",2013,Gordon Hall,altcoin_analysis
1,https://www.ccn.com/key-bitcoin-price-levels-w...,Key Bitcoin Price Levels for Week 51 (15 – 22 ...,"The Bitcoin price up to 2013/12/14, with Fib l...","<figure id=""attachment_3961"" style=""width: 838...",2013,Gordon Hall,altcoin_analysis
2,https://www.ccn.com/national-australia-bank-ci...,"National Australia Bank, Citing Highly Flawed ...",National Australia Bank looms over Melbourne’s...,"<figure id=""attachment_5011"" style=""width: 750...",2013,Gordon Hall,altcoin_analysis


In [4]:
# Convertimos el texto a tipo string para no obtener un error como el siguiente:
# AttributeError: 'float' object has no attribute 'lower'
# Este error se da en text = text.lower()

df.text = df.text.astype(str)

In [5]:
#Cleaning the text column
pat1 = r'http[^ ]+' # pattern to remove links
pat2 = r'www.[^ ]+' # pattern to remove links
pat3 = '[^a-zA-Z]+' # pattern to remove numbers
pat = r'|'.join((pat1, pat2, pat3))


#Cleaning data to get only representative words
def get_clean_text(text):
    text = text.lower() # lowercase
    text = re.sub(pat, ' ', text) # remove pat
    text = [w for w in tok.tokenize(text) if not w in stop_words] # remove stop_words
    text = [w for w in text if len(w)>1] # remove words with len <2
    text = (' '.join(text)).strip() # list to string
    return text

df['text_clean'] = [get_clean_text(t) for t in df['text']]

df.head(10)

,url,title,text,html,year,author,source,text_clean
0,https://www.ccn.com/bitcoin-price-update-will-...,Bitcoin Price Update: Will China Lead us Down?,Bitcoin Priced in USD on Mt. GoxAbove is Gox p...,"<figure id=""attachment_4090"" style=""width: 838...",2013,Gordon Hall,altcoin_analysis,bitcoin priced usd mt goxabove gox price pm gm...
1,https://www.ccn.com/key-bitcoin-price-levels-w...,Key Bitcoin Price Levels for Week 51 (15 – 22 ...,"The Bitcoin price up to 2013/12/14, with Fib l...","<figure id=""attachment_3961"" style=""width: 838...",2013,Gordon Hall,altcoin_analysis,bitcoin price fib levels dmas ok last week lik...
2,https://www.ccn.com/national-australia-bank-ci...,"National Australia Bank, Citing Highly Flawed ...",National Australia Bank looms over Melbourne’s...,"<figure id=""attachment_5011"" style=""width: 750...",2013,Gordon Hall,altcoin_analysis,national australia bank looms melbourne citize...
3,https://www.ccn.com/chinese-banking-ban-bitcoi...,Chinese Bitcoin Ban Driven by Chinese Banking...,Recent reports from China indicate there may b...,"<p><span style=""line-height: 1.5em;"">Recent re...",2013,Jonas Borchgrevink,altcoin_analysis,recent reports china indicate may bitcoin ban ...
4,https://www.ccn.com/bitcoin-trade-update-opene...,Bitcoin Trade Update: Opened Position,"Opened 1st Bitcoin trade position, price curre...","<figure id=""attachment_4921"" style=""width: 750...",2013,Gordon Hall,altcoin_analysis,opened st bitcoin trade position price current...
5,https://www.ccn.com/key-bitcoin-price-levels-w...,Key Bitcoin Price Levels for Week 52 (22 – 28 ...,"Gox Bitcoin Price, Volume, Fib Levels, trendli...","<figure id=""attachment_4520"" style=""width: 192...",2013,Gordon Hall,altcoin_analysis,gox bitcoin price volume fib levels trendline ...
6,https://www.ccn.com/bitcoin-survival/,Bitcoin Survival,Bitcoin was the means for Satoshi Nakamoto to ...,<p>Bitcoin was the means for Satoshi Nakamoto ...,2013,Paul Carrozzo,altcoin_analysis,bitcoin means satoshi nakamoto reward particip...
7,https://www.ccn.com/massive-bitcoin-sell-going/,Massive Bitcoin Sell Going On,Today we have seen a couple of massive sells t...,<p>Today we have seen a couple of massive sell...,2013,Jonas Borchgrevink,altcoin_analysis,today seen couple massive sells taking place m...
8,https://www.ccn.com/bitcoin-will-rise-monday-2...,Why Bitcoin will rise on Monday 23rd by more t...,Bitcoin has a tendency to spike on Monday and ...,<p>Bitcoin has a tendency to spike on Monday a...,2013,Jonas Borchgrevink,altcoin_analysis,bitcoin tendency spike monday tuesdays quiet w...
9,https://www.ccn.com/why-bitcoin-is-falling-and...,"Why Bitcoin is falling, and will rise again",Why Bitcoin is FallingThe Bitcoin price has ta...,"<figure id=""attachment_4086"" style=""width: 872...",2013,Jonas Borchgrevink,altcoin_analysis,bitcoin fallingthe bitcoin price taken heavy h...


In [6]:
ex = df['text_clean'][0]

print(ex)

bitcoin priced usd mt goxabove gox price pm gmt still previous low dipping retracement level note low volume leading us suspect trading fees implemented btc china documented elsewhere site fees still cut rate western standards chinese may taken negative sign believe free services common china highly competitive winner take economy bitcoin priced cny btc chinaand bitcoin price china note volume remains high bitcoin price appears weaker side believe btc china acting leading market setting bearish tone dragging exchanges note gox displays double top whereas btc china clear downwards trendline single peak write bleeding continued btc china price threatening previous low level things get serious big question chinese bitcoin price bounces gox side next support level bitcoin price fib previous low level certainly price support seen outside china bitcoin price undeniably pressure performs close key levels respective exchanges real test ready anything part cryptocurrency investment club cryptoc

In [7]:
def preprocess(sent):
    sent = nltk.word_tokenize(sent)
    sent = nltk.pos_tag(sent)
    return sent

sent = preprocess(ex)
sent

[('bitcoin', 'NN'),
 ('priced', 'VBN'),
 ('usd', 'JJ'),
 ('mt', 'NN'),
 ('goxabove', 'NN'),
 ('gox', 'NN'),
 ('price', 'NN'),
 ('pm', 'NN'),
 ('gmt', 'NN'),
 ('still', 'RB'),
 ('previous', 'JJ'),
 ('low', 'JJ'),
 ('dipping', 'VBG'),
 ('retracement', 'JJ'),
 ('level', 'NN'),
 ('note', 'NN'),
 ('low', 'JJ'),
 ('volume', 'NN'),
 ('leading', 'VBG'),
 ('us', 'PRP'),
 ('suspect', 'JJ'),
 ('trading', 'NN'),
 ('fees', 'NNS'),
 ('implemented', 'VBD'),
 ('btc', 'JJ'),
 ('china', 'NN'),
 ('documented', 'VBD'),
 ('elsewhere', 'RB'),
 ('site', 'NN'),
 ('fees', 'NNS'),
 ('still', 'RB'),
 ('cut', 'VB'),
 ('rate', 'NN'),
 ('western', 'JJ'),
 ('standards', 'NNS'),
 ('chinese', 'JJ'),
 ('may', 'MD'),
 ('taken', 'VBN'),
 ('negative', 'JJ'),
 ('sign', 'NN'),
 ('believe', 'VBP'),
 ('free', 'JJ'),
 ('services', 'NNS'),
 ('common', 'JJ'),
 ('china', 'VBP'),
 ('highly', 'RB'),
 ('competitive', 'JJ'),
 ('winner', 'NNS'),
 ('take', 'VBP'),
 ('economy', 'NN'),
 ('bitcoin', 'NN'),
 ('priced', 'VBN'),
 ('cny', 'NN

In [8]:
'''Our chunk pattern consists of one rule, that a noun phrase, NP,
should be formed whenever the chunker finds an optional determiner, DT,
followed by any number of adjectives, JJ, and then a noun, NN.'''

''''Fuente: 'https://towardsdatascience.com/named-entity-recognition-with-nltk-and-spacy-8c4a7d88e7da'
    Autor : 'Susan Li' '''

pattern = 'NP: {<DT>?<JJ>*<NN>}'

# Chunk parser

cp = nltk.RegexpParser(pattern)
cs = cp.parse(sent)
print(cs)

(S
  (NP bitcoin/NN)
  priced/VBN
  (NP usd/JJ mt/NN)
  (NP goxabove/NN)
  (NP gox/NN)
  (NP price/NN)
  (NP pm/NN)
  (NP gmt/NN)
  still/RB
  previous/JJ
  low/JJ
  dipping/VBG
  (NP retracement/JJ level/NN)
  (NP note/NN)
  (NP low/JJ volume/NN)
  leading/VBG
  us/PRP
  (NP suspect/JJ trading/NN)
  fees/NNS
  implemented/VBD
  (NP btc/JJ china/NN)
  documented/VBD
  elsewhere/RB
  (NP site/NN)
  fees/NNS
  still/RB
  cut/VB
  (NP rate/NN)
  western/JJ
  standards/NNS
  chinese/JJ
  may/MD
  taken/VBN
  (NP negative/JJ sign/NN)
  believe/VBP
  free/JJ
  services/NNS
  common/JJ
  china/VBP
  highly/RB
  competitive/JJ
  winner/NNS
  take/VBP
  (NP economy/NN)
  (NP bitcoin/NN)
  priced/VBN
  (NP cny/NN)
  (NP btc/NN)
  (NP chinaand/NN)
  (NP bitcoin/NN)
  (NP price/NN)
  china/NNS
  note/VBP
  (NP volume/NN)
  remains/VBZ
  (NP high/JJ bitcoin/NN)
  (NP price/NN)
  appears/VBZ
  weaker/JJR
  (NP side/NN)
  believe/VBP
  btc/IN
  (NP china/NN)
  acting/VBG
  leading/VBG
  (NP market/NN

In [9]:
from nltk.chunk import conlltags2tree, tree2conlltags
from pprint import pprint
iob_tagged = tree2conlltags(cs)
pprint(iob_tagged)

[('bitcoin', 'NN', 'B-NP'),
 ('priced', 'VBN', 'O'),
 ('usd', 'JJ', 'B-NP'),
 ('mt', 'NN', 'I-NP'),
 ('goxabove', 'NN', 'B-NP'),
 ('gox', 'NN', 'B-NP'),
 ('price', 'NN', 'B-NP'),
 ('pm', 'NN', 'B-NP'),
 ('gmt', 'NN', 'B-NP'),
 ('still', 'RB', 'O'),
 ('previous', 'JJ', 'O'),
 ('low', 'JJ', 'O'),
 ('dipping', 'VBG', 'O'),
 ('retracement', 'JJ', 'B-NP'),
 ('level', 'NN', 'I-NP'),
 ('note', 'NN', 'B-NP'),
 ('low', 'JJ', 'B-NP'),
 ('volume', 'NN', 'I-NP'),
 ('leading', 'VBG', 'O'),
 ('us', 'PRP', 'O'),
 ('suspect', 'JJ', 'B-NP'),
 ('trading', 'NN', 'I-NP'),
 ('fees', 'NNS', 'O'),
 ('implemented', 'VBD', 'O'),
 ('btc', 'JJ', 'B-NP'),
 ('china', 'NN', 'I-NP'),
 ('documented', 'VBD', 'O'),
 ('elsewhere', 'RB', 'O'),
 ('site', 'NN', 'B-NP'),
 ('fees', 'NNS', 'O'),
 ('still', 'RB', 'O'),
 ('cut', 'VB', 'O'),
 ('rate', 'NN', 'B-NP'),
 ('western', 'JJ', 'O'),
 ('standards', 'NNS', 'O'),
 ('chinese', 'JJ', 'O'),
 ('may', 'MD', 'O'),
 ('taken', 'VBN', 'O'),
 ('negative', 'JJ', 'B-NP'),
 ('sign', 'NN

In [10]:
ne_tree = ne_chunk(pos_tag(word_tokenize(ex)))
print(ne_tree)

(S
  bitcoin/NN
  priced/VBN
  usd/JJ
  mt/NN
  goxabove/NN
  gox/NN
  price/NN
  pm/NN
  gmt/NN
  still/RB
  previous/JJ
  low/JJ
  dipping/VBG
  retracement/JJ
  level/NN
  note/NN
  low/JJ
  volume/NN
  leading/VBG
  us/PRP
  suspect/JJ
  trading/NN
  fees/NNS
  implemented/VBD
  btc/JJ
  china/NN
  documented/VBD
  elsewhere/RB
  site/NN
  fees/NNS
  still/RB
  cut/VB
  rate/NN
  western/JJ
  standards/NNS
  chinese/JJ
  may/MD
  taken/VBN
  negative/JJ
  sign/NN
  believe/VBP
  free/JJ
  services/NNS
  common/JJ
  china/VBP
  highly/RB
  competitive/JJ
  winner/NNS
  take/VBP
  economy/NN
  bitcoin/NN
  priced/VBN
  cny/NN
  btc/NN
  chinaand/NN
  bitcoin/NN
  price/NN
  china/NNS
  note/VBP
  volume/NN
  remains/VBZ
  high/JJ
  bitcoin/NN
  price/NN
  appears/VBZ
  weaker/JJR
  side/NN
  believe/VBP
  btc/IN
  china/NN
  acting/VBG
  leading/VBG
  market/NN
  setting/VBG
  bearish/JJ
  tone/NN
  dragging/VBG
  exchanges/NNS
  note/VBP
  gox/NN
  displays/NNS
  double/RB
  top/JJ


In [11]:
df = df[['title', 'text_clean']]

df.head(5)

,title,text_clean
0,Bitcoin Price Update: Will China Lead us Down?,bitcoin priced usd mt goxabove gox price pm gm...
1,Key Bitcoin Price Levels for Week 51 (15 – 22 ...,bitcoin price fib levels dmas ok last week lik...
2,"National Australia Bank, Citing Highly Flawed ...",national australia bank looms melbourne citize...
3,Chinese Bitcoin Ban Driven by Chinese Banking...,recent reports china indicate may bitcoin ban ...
4,Bitcoin Trade Update: Opened Position,opened st bitcoin trade position price current...


In [12]:
def nlp(text):
    
    def preprocess(sent):

        sent = nltk.word_tokenize(sent)
        sent = nltk.pos_tag(sent)
        
        return sent

    sent = preprocess(ex)
    
    pattern = 'NP: {<DT>?<JJ>*<NN>}'

    # Chunk parser

    cp = nltk.RegexpParser(pattern)
    cs = cp.parse(sent)
    
    from nltk.chunk import conlltags2tree, tree2conlltags
    from pprint import pprint
    iob_tagged = tree2conlltags(cs)
    
    ne_tree = ne_chunk(pos_tag(word_tokenize(ex)))
    
    return dict(ne_tree)



In [13]:
nlp(df['text_clean'])

{'bitcoin': 'JJ',
 'priced': 'VBN',
 'usd': 'JJ',
 'mt': 'NN',
 'goxabove': 'NN',
 'gox': 'JJ',
 'price': 'NN',
 'pm': 'NN',
 'gmt': 'NN',
 'still': 'RB',
 'previous': 'JJ',
 'low': 'JJ',
 'dipping': 'VBG',
 'retracement': 'JJ',
 'level': 'NN',
 'note': 'VBP',
 'volume': 'NN',
 'leading': 'VBG',
 'us': 'PRP',
 'suspect': 'JJ',
 'trading': 'NN',
 'fees': 'NNS',
 'implemented': 'VBD',
 'btc': 'JJ',
 'china': 'JJ',
 'documented': 'VBD',
 'elsewhere': 'RB',
 'site': 'NN',
 'cut': 'VB',
 'rate': 'NN',
 'western': 'JJ',
 'standards': 'NNS',
 'chinese': 'JJ',
 'may': 'MD',
 'taken': 'VBN',
 'negative': 'JJ',
 'sign': 'NN',
 'believe': 'VBP',
 'free': 'JJ',
 'services': 'NNS',
 'common': 'JJ',
 'highly': 'RB',
 'competitive': 'JJ',
 'winner': 'NNS',
 'take': 'VBP',
 'economy': 'NN',
 'cny': 'NN',
 'chinaand': 'NN',
 'remains': 'VBZ',
 'high': 'JJ',
 'appears': 'VBZ',
 'weaker': 'JJR',
 'side': 'NN',
 'acting': 'VBG',
 'market': 'NN',
 'setting': 'VBG',
 'bearish': 'JJ',
 'tone': 'NN',
 'draggi

In [14]:
c = nlp(df['text_clean'])

vals = [x for x in c.values()]

set(vals)

{'CC',
 'IN',
 'JJ',
 'JJR',
 'MD',
 'NN',
 'NNS',
 'PRP',
 'RB',
 'VB',
 'VBD',
 'VBG',
 'VBN',
 'VBP',
 'VBZ'}

In [15]:
df.head()

,title,text_clean
0,Bitcoin Price Update: Will China Lead us Down?,bitcoin priced usd mt goxabove gox price pm gm...
1,Key Bitcoin Price Levels for Week 51 (15 – 22 ...,bitcoin price fib levels dmas ok last week lik...
2,"National Australia Bank, Citing Highly Flawed ...",national australia bank looms melbourne citize...
3,Chinese Bitcoin Ban Driven by Chinese Banking...,recent reports china indicate may bitcoin ban ...
4,Bitcoin Trade Update: Opened Position,opened st bitcoin trade position price current...


In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   title       500 non-null    object
 1   text_clean  500 non-null    object
dtypes: object(2)
memory usage: 7.9+ KB


In [17]:
ent_list = []

for i in range(len(df)):
    
    dictionary = nlp(df['text_clean'][i])

    vals = [x for x in dictionary.values()]
    
    # tot get rid of repeated elements, or you could later on count the amount of times they show up

    ent = set(vals)
    
    ent_list.append(ent)
    
    

In [18]:
df['entities'] = ent_list

df = df[['title','entities']]

df = df.rename(columns={'title':'document'})

df.head()

,document,entities
0,Bitcoin Price Update: Will China Lead us Down?,"{VBG, VBP, JJR, CC, VB, NN, PRP, VBN, NNS, JJ,..."
1,Key Bitcoin Price Levels for Week 51 (15 – 22 ...,"{VBG, VBP, JJR, CC, VB, NN, PRP, VBN, NNS, JJ,..."
2,"National Australia Bank, Citing Highly Flawed ...","{VBG, VBP, JJR, CC, VB, NN, PRP, VBN, NNS, JJ,..."
3,Chinese Bitcoin Ban Driven by Chinese Banking...,"{VBG, VBP, JJR, CC, VB, NN, PRP, VBN, NNS, JJ,..."
4,Bitcoin Trade Update: Opened Position,"{VBG, VBP, JJR, CC, VB, NN, PRP, VBN, NNS, JJ,..."
